<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [48]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [49]:
c.execute('DROP TABLE houses')

# Save (commit) the changes:
conn.commit()

In [50]:
c.execute('CREATE TABLE IF NOT EXISTS houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER)');
conn.commit()
print("Table created")



Table created


With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [51]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute("INSERT INTO houses VALUES (?, ?, ?, ?, ?)", last_sale)
# Remember to commit the changes
conn.commit()


Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [52]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
# 
c.executemany('INSERT INTO houses VALUES (?,?,?,?,?)', recent_sales)
print(c.fetchall())
conn.commit()

[]


Select all rows from houses

In [53]:
for row in c.execute('SELECT * FROM houses'):
    print(row)

(1, 4000, 5, 22, 619000)
(2, 2390, 4, 34, 319000)
(3, 1870, 3, 14, 289000)
(4, 1505, 3, 90, 269000)





#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [54]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('C:\Jeff\Office\IOD Course\Course\Lab 27May\housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()
type(data)
data

[[2104, 3, 70, 399900],
 [1600, 3, 28, 329900],
 [2400, 3, 44, 369000],
 [1416, 2, 49, 232000],
 [3000, 4, 75, 539900],
 [1985, 4, 61, 299900],
 [1534, 3, 12, 314900],
 [1427, 3, 57, 198999],
 [1380, 3, 14, 212000],
 [1494, 3, 15, 242500],
 [1940, 4, 7, 239999],
 [2000, 3, 27, 347000],
 [1890, 3, 45, 329999],
 [4478, 5, 49, 699900],
 [1268, 3, 58, 259900],
 [2300, 4, 77, 449900],
 [1320, 2, 62, 299900],
 [1236, 3, 78, 199900],
 [2609, 4, 5, 499998],
 [3031, 4, 21, 599000],
 [1767, 3, 44, 252900],
 [1888, 2, 79, 255000],
 [1604, 3, 13, 242900],
 [1962, 4, 53, 259900],
 [3890, 3, 36, 573900],
 [1100, 3, 60, 249900],
 [1458, 3, 29, 464500],
 [2526, 3, 13, 469000],
 [2200, 3, 28, 475000],
 [2637, 3, 25, 299900],
 [1839, 2, 40, 349900],
 [1000, 1, 5, 169900],
 [2040, 4, 75, 314900],
 [3137, 3, 67, 579900],
 [1811, 4, 24, 285900],
 [1437, 3, 50, 249900],
 [1239, 3, 22, 229900],
 [2132, 4, 28, 345000],
 [4215, 4, 66, 549000],
 [2162, 4, 43, 287000],
 [1664, 2, 40, 368500],
 [2238, 3, 37, 3299

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [55]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)
    print(d)

[None, 2104, 3, 70, 399900]
[None, 1600, 3, 28, 329900]
[None, 2400, 3, 44, 369000]
[None, 1416, 2, 49, 232000]
[None, 3000, 4, 75, 539900]
[None, 1985, 4, 61, 299900]
[None, 1534, 3, 12, 314900]
[None, 1427, 3, 57, 198999]
[None, 1380, 3, 14, 212000]
[None, 1494, 3, 15, 242500]
[None, 1940, 4, 7, 239999]
[None, 2000, 3, 27, 347000]
[None, 1890, 3, 45, 329999]
[None, 4478, 5, 49, 699900]
[None, 1268, 3, 58, 259900]
[None, 2300, 4, 77, 449900]
[None, 1320, 2, 62, 299900]
[None, 1236, 3, 78, 199900]
[None, 2609, 4, 5, 499998]
[None, 3031, 4, 21, 599000]
[None, 1767, 3, 44, 252900]
[None, 1888, 2, 79, 255000]
[None, 1604, 3, 13, 242900]
[None, 1962, 4, 53, 259900]
[None, 3890, 3, 36, 573900]
[None, 1100, 3, 60, 249900]
[None, 1458, 3, 29, 464500]
[None, 2526, 3, 13, 469000]
[None, 2200, 3, 28, 475000]
[None, 2637, 3, 25, 299900]
[None, 1839, 2, 40, 349900]
[None, 1000, 1, 5, 169900]
[None, 2040, 4, 75, 314900]
[None, 3137, 3, 67, 579900]
[None, 1811, 4, 24, 285900]
[None, 1437, 3, 50, 249

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [56]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [57]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()
print(data)

[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000], [None, 1416, 2, 49, 232000], [None, 3000, 4, 75, 539900], [None, 1985, 4, 61, 299900], [None, 1534, 3, 12, 314900], [None, 1427, 3, 57, 198999], [None, 1380, 3, 14, 212000], [None, 1494, 3, 15, 242500], [None, 1940, 4, 7, 239999], [None, 2000, 3, 27, 347000], [None, 1890, 3, 45, 329999], [None, 4478, 5, 49, 699900], [None, 1268, 3, 58, 259900], [None, 2300, 4, 77, 449900], [None, 1320, 2, 62, 299900], [None, 1236, 3, 78, 199900], [None, 2609, 4, 5, 499998], [None, 3031, 4, 21, 599000], [None, 1767, 3, 44, 252900], [None, 1888, 2, 79, 255000], [None, 1604, 3, 13, 242900], [None, 1962, 4, 53, 259900], [None, 3890, 3, 36, 573900], [None, 1100, 3, 60, 249900], [None, 1458, 3, 29, 464500], [None, 2526, 3, 13, 469000], [None, 2200, 3, 28, 475000], [None, 2637, 3, 25, 299900], [None, 1839, 2, 40, 349900], [None, 1000, 1, 5, 169900], [None, 2040, 4, 75, 314900], [None, 3137, 3, 67, 579900], [None, 1811, 4, 

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [58]:
import numpy as np
d1 = np.array([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()
#for row in houses:
 #   print(row)

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [59]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do?

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [60]:
# ANSWER
bdrm4 = c.execute("SELECT * FROM houses WHERE bdrms='4'")
bdrm4.fetchall()


[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [61]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1827.5555555555557, 322892.51851851854),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [62]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [63]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('C:\Jeff\Office\IOD Course\Course\Lab 27May\housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [64]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)


Run a query to get the average price of each house size from this table:

In [65]:
#ANSWER
average_price = c.execute("SELECT bdrms, AVG(price) AS avg_price FROM houses_pandas GROUP BY bdrms")
average_price.fetchall()

[(1, 169900.0),
 (2, 280866.6666666667),
 (3, 326403.92),
 (4, 377449.78571428574),
 (5, 699900.0)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [66]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [67]:
# Reading CSV to Dataframe
orders = pd.read_csv('C:\Jeff\Office\IOD Course\Course\Lab 27May\P12-ListOfOrders.csv', low_memory = False, encoding = 'utf-8')
orders_break_down =  pd.read_csv('C:\Jeff\Office\IOD Course\Course\Lab 27May\P12-OrderBreakdown.csv', low_memory = False, encoding = 'utf-8')

In [68]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [69]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [70]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]
orders.head()

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [71]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]
orders_break_down.head()

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 3. Check DataTypes 

In [72]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [73]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [74]:
# Establishing Local DB connection
db_connection = sqlite3.connect('eshop.db.sqlite')
# Save these two dataframes as a table in sqlite
c = db_connection.cursor()
#orders.to_sql('orders', con = conn, if_exists = 'replace', index = False)
db_connection.commit()
orders.to_sql('orders', con = db_connection, if_exists = 'replace', index = False)
orders_break_down.to_sql('orders_break_down', con = db_connection, if_exists = 'replace', index = False)

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [75]:
# Select first 10 rows of all columns in orders table
query = """
         SELECT * 
         FROM orders 
         LIMIT 10;
         """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [76]:
# Select first 10 rows of all columns in orders table
query = """
         SELECT * 
         FROM orders_break_down 
         LIMIT 10;
         """

orders_break_down_head_10 = sql.read_sql(query, con=db_connection)
orders_break_down_head_10

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies
5,ES-2011-3848439,"Binney & Smith Pencil Sharpener, Water Color",0.0,90.0,3,Office Supplies
6,ES-2011-3848439,"Sanford Canvas, Fluorescent",0.0,207.0,4,Office Supplies
7,ES-2011-5433855,"Bush Floating Shelf Set, Pine",0.1,155.0,1,Furniture
8,ES-2011-5433855,"Accos Thumb Tacks, Assorted Sizes",0.0,33.0,3,Office Supplies
9,ES-2011-5433855,"Smead Lockers, Industrial",0.1,716.0,4,Office Supplies


In [77]:
# Select Number of Orders for Each Customer
orders_customer = c.execute("SELECT order_id, customer_name FROM orders GROUP BY customer_name")
orders_customer = c.fetchall()
orders_customer

[('ES-2011-3528133', 'Aaron Bergman'),
 ('IT-2011-4320455', 'Aaron Hawkins'),
 ('ES-2011-5083176', 'Aaron Smayling'),
 ('ES-2012-1205030', 'Adam Bellavance'),
 ('IT-2011-4648014', 'Adam Hart'),
 ('IT-2011-1521406', 'Adam Shillingsburg'),
 ('ES-2012-4263971', 'Adrian Barton'),
 ('ES-2012-3241112', 'Adrian Hane'),
 ('IT-2012-4297670', 'Adrian Shami'),
 ('ES-2011-4880899', 'Aimee Bixby'),
 ('ES-2011-5118812', 'Alan Barnes'),
 ('IT-2012-3719538', 'Alan Dominguez'),
 ('ES-2011-1761215', 'Alan Haines'),
 ('IT-2011-5808077', 'Alan Hwang'),
 ('ES-2012-2102738', 'Alan Schoenberger'),
 ('ES-2011-4919091', 'Alan Shonely'),
 ('ES-2011-5441485', 'Alejandro Ballentine'),
 ('ES-2013-1265335', 'Alejandro Grove'),
 ('ES-2012-1071143', 'Alejandro Savely'),
 ('ES-2012-3866726', 'Aleksandra Gannaway'),
 ('ES-2012-2960821', 'Alex Avila'),
 ('IT-2011-2226067', 'Alex Grayson'),
 ('ES-2012-1678064', 'Alex Russell'),
 ('IT-2011-2419493', 'Alice McCarthy'),
 ('ES-2013-2061669', 'Allen Armold'),
 ('IT-2012-51414

#### 6. Select Number of Customers for Each Country

In [78]:
# Select Number of Customers for Each Country
num_customers = c.execute("SELECT COUNT(customer_name),country FROM orders GROUP BY country")
num_customers = c.fetchall()
num_customers

[(135, 'Austria'),
 (68, 'Belgium'),
 (29, 'Denmark'),
 (34, 'Finland'),
 (991, 'France'),
 (806, 'Germany'),
 (50, 'Ireland'),
 (493, 'Italy'),
 (194, 'Netherlands'),
 (37, 'Norway'),
 (37, 'Portugal'),
 (403, 'Spain'),
 (100, 'Sweden'),
 (40, 'Switzerland'),
 (700, 'United Kingdom')]

#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [79]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table
dis_sal_qua = c.execute("SELECT order_id, discount, sales, quantity FROM orders_break_down")
dis_sal_qua = c.fetchall()
dis_sal_qua

[('IT-2011-3647632', 0.5, 45.0, 3),
 ('ES-2011-4869686', 0.0, 854.0, 7),
 ('ES-2011-4939443', 0.0, 140.0, 3),
 ('IT-2011-2942451', 0.5, 27.0, 2),
 ('IT-2011-2942451', 0.5, 17.0, 2),
 ('ES-2011-3848439', 0.0, 90.0, 3),
 ('ES-2011-3848439', 0.0, 207.0, 4),
 ('ES-2011-5433855', 0.1, 155.0, 1),
 ('ES-2011-5433855', 0.0, 33.0, 3),
 ('ES-2011-5433855', 0.1, 716.0, 4),
 ('IT-2011-4546695', 0.6, 987.0, 6),
 ('IT-2011-4546695', 0.5, 116.0, 5),
 ('ES-2011-5158390', 0.0, 1384.0, 3),
 ('ES-2011-5158390', 0.0, 103.0, 2),
 ('ES-2011-4359424', 0.6, 268.0, 2),
 ('ES-2011-1466305', 0.0, 55.0, 1),
 ('ES-2011-1466305', 0.0, 97.0, 2),
 ('ES-2011-4359424', 0.0, 40.0, 5),
 ('ES-2011-1138719', 0.0, 22.0, 3),
 ('ES-2011-5040255', 0.5, 34.0, 2),
 ('ES-2011-4976144', 0.0, 290.0, 5),
 ('ES-2011-1460199', 0.1, 552.0, 5),
 ('ES-2011-1460199', 0.0, 257.0, 5),
 ('ES-2011-1460199', 0.0, 17.0, 2),
 ('ES-2011-1460199', 0.0, 522.0, 2),
 ('IT-2011-5340302', 0.5, 9.0, 3),
 ('IT-2011-5340302', 0.5, 22.0, 4),
 ('IT-2011-534

#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [80]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table
total_price = c.execute("SELECT order_id, discount, sales, quantity, ((sales * quantity)-discount) AS total_price FROM orders_break_down")
total_price = c.fetchall()
total_price

[('IT-2011-3647632', 0.5, 45.0, 3, 134.5),
 ('ES-2011-4869686', 0.0, 854.0, 7, 5978.0),
 ('ES-2011-4939443', 0.0, 140.0, 3, 420.0),
 ('IT-2011-2942451', 0.5, 27.0, 2, 53.5),
 ('IT-2011-2942451', 0.5, 17.0, 2, 33.5),
 ('ES-2011-3848439', 0.0, 90.0, 3, 270.0),
 ('ES-2011-3848439', 0.0, 207.0, 4, 828.0),
 ('ES-2011-5433855', 0.1, 155.0, 1, 154.9),
 ('ES-2011-5433855', 0.0, 33.0, 3, 99.0),
 ('ES-2011-5433855', 0.1, 716.0, 4, 2863.9),
 ('IT-2011-4546695', 0.6, 987.0, 6, 5921.4),
 ('IT-2011-4546695', 0.5, 116.0, 5, 579.5),
 ('ES-2011-5158390', 0.0, 1384.0, 3, 4152.0),
 ('ES-2011-5158390', 0.0, 103.0, 2, 206.0),
 ('ES-2011-4359424', 0.6, 268.0, 2, 535.4),
 ('ES-2011-1466305', 0.0, 55.0, 1, 55.0),
 ('ES-2011-1466305', 0.0, 97.0, 2, 194.0),
 ('ES-2011-4359424', 0.0, 40.0, 5, 200.0),
 ('ES-2011-1138719', 0.0, 22.0, 3, 66.0),
 ('ES-2011-5040255', 0.5, 34.0, 2, 67.5),
 ('ES-2011-4976144', 0.0, 290.0, 5, 1450.0),
 ('ES-2011-1460199', 0.1, 552.0, 5, 2759.9),
 ('ES-2011-1460199', 0.0, 257.0, 5, 1285.

#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [81]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100
all_orders = c.execute("SELECT order_id, ((sales * quantity)-discount) AS total_price FROM orders_break_down WHERE ((sales * quantity)-discount) >100")
all_orders = c.fetchall()
all_orders

[('IT-2011-3647632', 134.5),
 ('ES-2011-4869686', 5978.0),
 ('ES-2011-4939443', 420.0),
 ('ES-2011-3848439', 270.0),
 ('ES-2011-3848439', 828.0),
 ('ES-2011-5433855', 154.9),
 ('ES-2011-5433855', 2863.9),
 ('IT-2011-4546695', 5921.4),
 ('IT-2011-4546695', 579.5),
 ('ES-2011-5158390', 4152.0),
 ('ES-2011-5158390', 206.0),
 ('ES-2011-4359424', 535.4),
 ('ES-2011-1466305', 194.0),
 ('ES-2011-4359424', 200.0),
 ('ES-2011-4976144', 1450.0),
 ('ES-2011-1460199', 2759.9),
 ('ES-2011-1460199', 1285.0),
 ('ES-2011-1460199', 1044.0),
 ('IT-2011-5340302', 395.5),
 ('IT-2011-5951216', 154.5),
 ('ES-2011-1043483', 2464.0),
 ('ES-2011-1043483', 1458.0),
 ('ES-2011-1043483', 2640.0),
 ('ES-2011-3661686', 248.9),
 ('ES-2011-3661686', 850.0),
 ('ES-2011-3661686', 110.0),
 ('ES-2011-3661686', 1054.0),
 ('ES-2011-4174897', 864.0),
 ('ES-2011-3666932', 435.0),
 ('ES-2011-3666932', 143.6),
 ('ES-2011-1690587', 144.0),
 ('ES-2011-1122594', 116.0),
 ('ES-2011-3229667', 990.0),
 ('ES-2011-5497239', 1650.0),
 

#### 8. Select All Customer And The Product They Have Bought

In [82]:
# Select All Customer And The Product They Have Bought
all_customers = c.execute("SELECT orders.customer_name, orders_break_down.product_name FROM orders INNER JOIN orders_break_down ON orders.order_id=orders_break_down.order_id")
all_customers = c.fetchall()
all_customers

[('Eugene Moren', 'Enermax Note Cards, Premium'),
 ('Dorothy Dickinson', 'Dania Corner Shelving, Traditional'),
 ('Arthur Prichep', 'Binney & Smith Sketch Pad, Easy-Erase'),
 ('Grant Thornton', 'Boston Markers, Easy-Erase'),
 ('Grant Thornton', 'Eldon Folders, Single Width'),
 ('Michael Granlund', 'Binney & Smith Pencil Sharpener, Water Color'),
 ('Michael Granlund', 'Sanford Canvas, Fluorescent'),
 ('Dave Poirier', 'Accos Thumb Tacks, Assorted Sizes'),
 ('Dave Poirier', 'Bush Floating Shelf Set, Pine'),
 ('Dave Poirier', 'Smead Lockers, Industrial'),
 ('Darren Powers', 'Binney & Smith Sketch Pad, Blue'),
 ('Darren Powers', 'Ikea Classic Bookcase, Metal'),
 ('Eric Murdock', 'Ibico Hole Reinforcements, Recycled'),
 ('Mick Brown', 'Boston Canvas, Fluorescent'),
 ('Mick Brown', 'Smead Trays, Single Width'),
 ('Dorothy Dickinson', 'Bevis Training Table, with Bottom Storage'),
 ('Dorothy Dickinson', 'Novimex File Folder Labels, Alphabetical'),
 ('Roy Collins', 'Binney & Smith Canvas, Blue')

#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [83]:
# Select Number of 'Furniture' Orders For Each Country
num_furn = c.execute("""
SELECT orders.country, 
COUNT(orders.country) FROM orders INNER JOIN orders_break_down ON orders.order_id=orders_break_down.order_id 
WHERE (orders_break_down.category)='Furniture' 
GROUP BY orders.country 
ORDER BY orders_break_down.category""")

num_furn = c.fetchall()
num_furn

[('Austria', 40),
 ('Belgium', 20),
 ('Denmark', 9),
 ('Finland', 16),
 ('France', 299),
 ('Germany', 264),
 ('Ireland', 12),
 ('Italy', 136),
 ('Netherlands', 76),
 ('Norway', 6),
 ('Portugal', 10),
 ('Spain', 123),
 ('Sweden', 36),
 ('Switzerland', 11),
 ('United Kingdom', 180)]

#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [89]:
# Select Number of 'Furniture' Orders For The Country Denmark
num_furn_den = c.execute("""
SELECT od.country, COUNT(od.country) 
FROM orders od INNER JOIN orders_break_down obd 
ON od.order_id=obd.order_id 
WHERE (obd.category)='Furniture' AND (od.country)='Denmark' 
ORDER BY obd.category""")

num_furn_den = c.fetchall()
num_furn_den

[('Denmark', 9)]

#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [102]:
# Select Total Sales With Discount and Without Discount for Each Country
dis_wo = c.execute("""
SELECT od.country,
SUM((obd.sales * obd.quantity) - obd.discount) AS total_price_discounted,
SUM(obd.sales * obd.quantity) AS wo_discount
FROM orders od INNER JOIN orders_break_down obd 
ON od.order_id=obd.order_id 
GROUP BY od.country
ORDER BY obd.category""")

dis_wo.fetchall()



[('United Kingdom', 2161854.799999994, 2161947.0),
 ('Austria', 391694.0, 391694.0),
 ('Belgium', 229829.0, 229829.0),
 ('Denmark', 34586.50000000001, 34617.0),
 ('Finland', 82001.0, 82001.0),
 ('France', 3108444.9999999907, 3108592.0),
 ('Germany', 2411393.149999979, 2411483.0),
 ('Italy', 1177798.4000000004, 1177921.0),
 ('Netherlands', 379525.09999999974, 379714.0),
 ('Norway', 99878.0, 99878.0),
 ('Spain', 1340805.8999999969, 1340832.0),
 ('Sweden', 168357.89999999997, 168461.0),
 ('Switzerland', 141661.0, 141661.0),
 ('Ireland', 75673.70000000001, 75724.0),
 ('Portugal', 94594.0, 94629.0)]

In [103]:
query = """
         SELECT * 
         FROM orders 
         LIMIT 10;
         """

orders_10 = sql.read_sql(query, con=db_connection)
orders_10


,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [104]:
query = """
         SELECT * 
         FROM orders_break_down 
         LIMIT 10;
         """

orders_break_down_head_10 = sql.read_sql(query, con=db_connection)
orders_break_down_head_10


,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies
5,ES-2011-3848439,"Binney & Smith Pencil Sharpener, Water Color",0.0,90.0,3,Office Supplies
6,ES-2011-3848439,"Sanford Canvas, Fluorescent",0.0,207.0,4,Office Supplies
7,ES-2011-5433855,"Bush Floating Shelf Set, Pine",0.1,155.0,1,Furniture
8,ES-2011-5433855,"Accos Thumb Tacks, Assorted Sizes",0.0,33.0,3,Office Supplies
9,ES-2011-5433855,"Smead Lockers, Industrial",0.1,716.0,4,Office Supplies


#### 11.A Select Total Quantity, Total Sales for Each Country

In [114]:
# Select Total Quantity, Total Sales for Each Country
qty_sal = c.execute("""
SELECT od.country, SUM(obd.quantity), SUM(obd.sales)
FROM orders od INNER JOIN orders_break_down obd 
ON od.order_id=obd.order_id 
GROUP BY od.country
""")

qty_sal.fetchall()


[('Austria', 973, 79382.0),
 ('Belgium', 532, 42320.0),
 ('Denmark', 204, 7763.0),
 ('Finland', 201, 20702.0),
 ('France', 7329, 609683.0),
 ('Germany', 6179, 488681.0),
 ('Ireland', 392, 15998.0),
 ('Italy', 3612, 252742.0),
 ('Netherlands', 1526, 70313.0),
 ('Norway', 261, 20529.0),
 ('Portugal', 286, 15106.0),
 ('Spain', 2881, 249402.0),
 ('Sweden', 753, 30490.0),
 ('Switzerland', 308, 24874.0),
 ('United Kingdom', 4917, 420497.0)]

#### 11.B Select Top 3 Country Based on Sales

In [119]:
# Select Top 3 Country Based on Sales
top_3country = c.execute("""
SELECT od.country, SUM(obd.sales) AS total_sales
FROM orders od INNER JOIN orders_break_down obd 
ON od.order_id=obd.order_id 
GROUP BY od.country
ORDER BY total_sales DESC
LIMIT 3
""")

top_3country.fetchall()


[('France', 609683.0), ('Germany', 488681.0), ('United Kingdom', 420497.0)]

#### 11.C Select Bottom 3 Country Based On Quantities

In [120]:
# Select Bottom 3 Country Based On Quantities
bottom_3country = c.execute("""
SELECT od.country, SUM(obd.quantity) AS total_quantity
FROM orders od INNER JOIN orders_break_down obd 
ON od.order_id=obd.order_id 
GROUP BY od.country
ORDER BY total_quantity ASC
LIMIT 3
""")

bottom_3country.fetchall()


[('Finland', 201), ('Denmark', 204), ('Norway', 261)]

#### 12. Select Average Sales By Categroy For The Country 'France'

In [123]:
# Select Average Sales By Categroy For The Country 'France'

avr_sal_cat = c.execute("""
SELECT od.country, AVG(obd.sales) AS sales_average, obd.category
FROM orders od INNER JOIN orders_break_down obd 
ON od.order_id=obd.order_id 
WHERE od.country = 'France'
GROUP BY obd.category
ORDER BY sales_average
""")

avr_sal_cat.fetchall()


[('France', 167.21770917952884, 'Office Supplies'),
 ('France', 582.314381270903, 'Furniture'),
 ('France', 595.1450777202073, 'Technology')]

#### 13. Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [135]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest
ccts = c.execute("""
SELECT od.country, SUM(obd.sales) AS tot_sales, obd.category
FROM orders od INNER JOIN orders_break_down obd 
ON od.order_id=obd.order_id 
GROUP BY od.country
ORDER BY tot_sales DESC
""")

ccts.fetchall()


[('France', 609683.0, 'Office Supplies'),
 ('Germany', 488681.0, 'Office Supplies'),
 ('United Kingdom', 420497.0, 'Furniture'),
 ('Italy', 252742.0, 'Office Supplies'),
 ('Spain', 249402.0, 'Office Supplies'),
 ('Austria', 79382.0, 'Office Supplies'),
 ('Netherlands', 70313.0, 'Office Supplies'),
 ('Belgium', 42320.0, 'Office Supplies'),
 ('Sweden', 30490.0, 'Office Supplies'),
 ('Switzerland', 24874.0, 'Office Supplies'),
 ('Finland', 20702.0, 'Office Supplies'),
 ('Norway', 20529.0, 'Office Supplies'),
 ('Ireland', 15998.0, 'Technology'),
 ('Portugal', 15106.0, 'Technology'),
 ('Denmark', 7763.0, 'Office Supplies')]

### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



